# Install

In [1]:
# cuda 11.1
#!pip install -r requirements-torch-cu111.txt --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111

# Data preperation

In [111]:
prefix = '1cycle'
url = 'https://drive.google.com/uc?id=1j46elyFZtkmnmCehlntMi0eX0Tp5nnav'
#prefix = 'helper'
#url = 'https://drive.google.com/uc?id=1iSP_YKFs56d5cRRTEMzfedwRxrx-nXWO'

## 스토리헬퍼 샘플 데이터 다운로드

In [112]:
import gdown

scripts_file = f'data/scripts_{prefix}.json'
gdown.download(url, scripts_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1j46elyFZtkmnmCehlntMi0eX0Tp5nnav
To: /home/kotech/workspace/ScriptWriter/data/scripts_1cycle.json
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12.6M/12.6M [00:03<00:00, 3.49MB/s]


'data/scripts_1cycle.json'

In [3]:
import json

with open(scripts_file) as f:
    data_dict = json.load(f)

In [4]:
# 샘플 데이터 출력
data_dict[0]

{'storyline': '우연히 옆집으로 이사 오게 된 차우와 수리첸',
 'script': ['집을 구하는 차우.',
  '그러나 수엔 부인은 방이 나갔다는 것을 알린다.',
  '수엔 부인은 옆집에도 방이 있다며 한 번 볼 것을 권한다.',
  '이사를 하는 차우와 수리첸.',
  '그러나 동시에 이사를 해서 짐들이 계속 섞인다.',
  '남편은 어디에 있냐고 묻는 수엔 부인.',
  '수리첸은 남편이 출장 중이라고 말한다.',
  '수리첸에게 책을 건네는 차우']}

**후처리**
1. `\n`을 제거한다. "부엌에서 일하게 된 마리오\n인부들 사이에서 인기만점인 베아트리체"  
   ==> 필요없는 것 같음.

In [5]:
# 비정상적 white character가 없는지 확인
for idx, data in enumerate(data_dict):
    #data['storyline'] = data['storyline'].replace('\n', ' ')
    for i, context in enumerate(data['script']):
        #if '\n' in context:
        if '부엌에서 일하게' in context:
            print(idx, i, context)
            print('"%s%s"'%(context[9],context[10]))
            print(context[10] == ' ')
        #data['script'][i] = context.replace('\n', ' ')

1378 7 부엌에서 일하게 된 마리오
인부들 사이에서 인기만점인 베아트리체
"된 "
True


## Tokenizer

### kobigbird pretrained model을 이용한 tokenize

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('monologg/kobigbird-bert-base')

In [7]:
# tokenizer 기본 설정 정보 출력
print(tokenizer)
print()
# tokenizer special token정보 출력
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print('sep_token_id =', tokenizer.sep_token_id)
print('unk_token_id =', tokenizer.unk_token_id)

PreTrainedTokenizerFast(name_or_path='monologg/kobigbird-bert-base', vocab_size=32500, model_max_len=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

['<s>', '</s>', '[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[5, 6, 1, 3, 0, 2, 4]
sep_token_id = 3
unk_token_id = 1


In [8]:
s = data_dict[0]['storyline']
print(s)
tokens = tokenizer.tokenize(s)
print(tokens)
encoded = tokenizer(s)
print(encoded)
print()
output = tokenizer.convert_tokens_to_string(tokens)
print(output)
decoded = tokenizer.decode(encoded['input_ids'])
print(decoded)
print()

# [CLS]와 [SEP]은 필요없으므로....
encoded = tokenizer(s, add_special_tokens=False)
decoded = tokenizer.decode(encoded['input_ids'])
print(decoded)
print()

# [SEP] token을 ScriptWriter모델의 EOS로 사용하자
print(tokenizer)
print(tokenizer.tokenize('[SEP]'))

우연히 옆집으로 이사 오게 된 차우와 수리첸
['우연히', '옆집', '##으로', '이사', '오', '##게', '된', '차우', '##와', '수리', '##첸']
{'input_ids': [2, 12281, 23393, 9627, 7384, 3649, 4696, 2936, 27898, 4756, 9869, 5810, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

우연히 옆집으로 이사 오게 된 차우와 수리첸
[CLS] 우연히 옆집으로 이사 오게 된 차우와 수리첸 [SEP]

우연히 옆집으로 이사 오게 된 차우와 수리첸

PreTrainedTokenizerFast(name_or_path='monologg/kobigbird-bert-base', vocab_size=32500, model_max_len=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['[SEP]']


# source code 분석

### 학습 및 검증, 테스트 데이터 셋 생성

```py
def generate_data_with_random_samples():
    # generate negative samples randomly
    # In training set, for each sample, we randomly sample a response as a negative candidate
    # In development and test set, for each sample, we randomly sample 9 responses as negative candidates and we add a "EOS" response as a candidate to let model select when to stop
```
무작위로 negative sample을 생성한다.  
학습set에서는 한개의 무작위 sample을 negative 후보로서 생성한다.
```py
        for context_id, narrative_id, _ in train:
            num_context = len(context_id)
            for i in range(1, num_context):
                context = context_id[:i]
                response = context_id[i]
                train_all.append([context, response, narrative_id, 1])
                flag = True
                while flag:
                    # 무작위로 스토리 중의 하나를 고르고, 그 중 무작위 script를 골라서 negative 샘플을 생성한다.
                    random_idx = random.randint(0, len(positive_data) - 1)
                    random_context = positive_data[random_idx][0]
                    random_idx_2 = random.randint(0, len(random_context) - 1)
                    random_response = random_context[random_idx_2]
                    # 만일 정답 response와 추출한 response의 길이가 다르다면 negative 샘플로 추가한다.
                    if len(response) != len(random_response):
                        flag = False
                        train_all.append([context, random_response, narrative_id, 0])
                    # 길이가 같다면, 내용을 비교해서 다르다면 negative 샘플로 추가한다. 
                    else:
                        for idx, wid in enumerate(response):
                            if wid != random_response[idx]:
                                flag = False
                                train_all.append([context, random_response, narrative_id, 0])
                                break
```
검증set에서는 9개의 negative sample을 추가한다.  
response가 EOS인 경우(마지막인 경우), EOS를 True로 추가한다. (중복???)  
그렇지 않을 경우, EOS response를 negative(False)로 추가한다.  
테스트set도 동일하다.  
```py
        for context_id, narrative_id, _ in dev:
            num_context = len(context_id)
            for i in range(1, num_context):
                context = context_id[:i]
                response = context_id[i]
                dev_all.append([context, response, narrative_id, 1])
                count = 0
                negative_samples = []
                # total count = 1(positive) + 8(negative) + 1(EOS) 가 되어야함.
                # 아래의 count는 count < 8로 수정 필요
                while count < 9:
                    random_idx = random.randint(0, len(positive_data) - 1)
                    random_context = positive_data[random_idx][0]
                    random_idx_2 = random.randint(0, len(random_context) - 1)
                    random_response = random_context[random_idx_2]
                    if random_response not in negative_samples and random_response != [EOS_ID]:
                        if len(response) != len(random_response):
                            dev_all.append([context, random_response, narrative_id, 0])
                            count += 1
                            negative_samples.append(random_response)
                        else:
                            for idx, wid in enumerate(response):
                                if wid != random_response[idx]:
                                    dev_all.append([context, random_response, narrative_id, 0])
                                    count += 1
                                    negative_samples.append(random_response)
                                    break
                if response == [EOS_ID]:
                    dev_all.append([context, [EOS_ID], narrative_id, 1])
                else:
                    dev_all.append([context, [EOS_ID], narrative_id, 0])
```
**To-Do**   
Solr sample 추가가 필요한가?  

### pickle data 분석

In [9]:
import pickle

with open('data/train.gr.pkl', 'rb') as f:
    train_gr = pickle.load(f)
    
with open('data/dev.gr.pkl', 'rb') as f:
    dev_gr = pickle.load(f)
    
with open('data/test.gr.pkl', 'rb') as f:
    test_gr = pickle.load(f)
    
with open('data/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

In [10]:
print('train data tuple size =', len(train_gr))
print('utterance shape =', train_gr[0].shape)
print('response shape =', train_gr[1].shape)
print('narrative shape =', train_gr[2].shape)
print('gt_response shape =', train_gr[3].shape)
print('y_true shape =', train_gr[4].shape)

train data tuple size = 5
utterance shape = (136524, 11, 50)
response shape = (136524, 50)
narrative shape = (136524, 50)
gt_response shape = (136524, 50)
y_true shape = (136524,)


In [11]:
train_number = len(train_gr[4])
val_number = len(dev_gr[4])
test_number = len(test_gr[4])
#total_number_of_data = 

In [12]:
# original dataset numbers
train_number, val_number, test_number

(136524, 37480, 38320)

In [13]:
def get_data(data, idx):
    return data[0][idx], data[1][idx], data[2][idx], data[3][idx], data[4][idx]

def sample_data_print(utterance, response, narrative, gt_response, y_true):
    #print('1st script =', utterance[0])
    #print('2nd script =', utterance[1])
    for i, utt in enumerate(utterance):
        if (utt == [0]*50).all():
            break
    print('utterance cnt =', i)
    print('resonse =', response)
    print('narrative =', narrative)
    print('gt_response =', gt_response)
    print('y_true =', y_true)

def browse_idx(data, idx):
    utterance, response, narrative, gt_response, y_true = get_data(data, idx)
    sample_data_print(utterance, response, narrative, gt_response, y_true)

def get_utterance_len(data, idx):
    utterance, response, narrative, gt_response, y_true = get_data(data, idx)
    for i, utt in enumerate(utterance):
        if (utt == [0]*50).all():
            break
    return i

browse_idx(train_gr, 0)

utterance cnt = 1
resonse = [   5  164   48    4 5334  123 2778    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
narrative = [10958 13504   137     8  2778     4     3    11    29   267     8     9
    11 10915   867  3205     1    46   101   111  4745 23556   867     3
   173    29    99  2373  2523    51     2    64     3     8    47    51
    72  5339 16837     2     1   104  4531    27     2   363    15   117
    71  5440]
gt_response = [   5  164   48    4 5334  123 2778    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
y_true = 1


In [14]:
# negative sample
browse_idx(train_gr, 1)

utterance cnt = 1
resonse = [   5   73   31   57  130    9 4446    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
narrative = [10958 13504   137     8  2778     4     3    11    29   267     8     9
    11 10915   867  3205     1    46   101   111  4745 23556   867     3
   173    29    99  2373  2523    51     2    64     3     8    47    51
    72  5339 16837     2     1   104  4531    27     2   363    15   117
    71  5440]
gt_response = [   5  164   48    4 5334  123 2778    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
y_true = 0


In [15]:
# 2nd sample with two utterance
browse_idx(train_gr, 2)

utterance cnt = 2
resonse = [267  77  10   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
narrative = [10958 13504   137     8  2778     4     3    11    29   267     8     9
    11 10915   867  3205     1    46   101   111  4745 23556   867     3
   173    29    99  2373  2523    51     2    64     3     8    47    51
    72  5339 16837     2     1   104  4531    27     2   363    15   117
    71  5440]
gt_response = [267  77  10   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
y_true = 1


In [16]:
# last script
browse_idx(train_gr, 14)

utterance cnt = 8
resonse = [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
narrative = [10958 13504   137     8  2778     4     3    11    29   267     8     9
    11 10915   867  3205     1    46   101   111  4745 23556   867     3
   173    29    99  2373  2523    51     2    64     3     8    47    51
    72  5339 16837     2     1   104  4531    27     2   363    15   117
    71  5440]
gt_response = [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_true = 1


In [17]:
train_gr[0].shape[0]

136524

In [18]:
# check utterance len
max_utt_len = 0
for i in range(train_gr[0].shape[0]):
    utt_len = get_utterance_len(train_gr, i)
    if utt_len > max_utt_len:
        max_utt_len = utt_len
        print(i, max_utt_len)

0 1
2 2
4 3
6 4
8 5
10 6
12 7
14 8
120 9
314 10


In [19]:
# check utterance len
max_utt_len = 0
for i in range(dev_gr[0].shape[0]):
    utt_len = get_utterance_len(dev_gr, i)
    if utt_len > max_utt_len:
        max_utt_len = utt_len
        print(i, max_utt_len)

0 1
10 2
50 3
60 4
110 5
470 6
480 7
760 8
850 9
1300 10


In [20]:
# 최대 10개의 utterance가 있다. 11번째는 항상 [0.]*50이고, [EOS_ID]는 없을 수도 있다.
utterance, response, narrative, gt_response, y_true = get_data(dev_gr, 1300)
utterance[-3:]

array([[  486,    32,    73,  9178,     5,     2, 11099,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [  913, 15855,     2,   128,   105,   169,    37,  7022,     2,
         4885,  9643,     2,  2914,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,    

In [21]:
# 최대 10개의 utterance가 있다. 11번째는 항상 [0.]*50이고, [EOS_ID]는 없을 수도 있다.
utterance, response, narrative, gt_response, y_true = get_data(train_gr, 314)
utterance[-3:]

array([[ 1758,  7785,   183,    15,    11,  1060,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [ 1856,     1,     6,    33,  2433, 11766,     2,  3945,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,    

In [22]:
utterance, response, narrative, gt_response, y_true = get_data(train_gr, 312)
utterance[-3:]

array([[1758, 7785,  183,   15,   11, 1060,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   

In [23]:
# 정답이 EOS인 경우 10번째 뒤에 중복적으로 데이터 들어 있음
browse_idx(dev_gr, 10)
print()
browse_idx(dev_gr, 19)

utterance cnt = 2
resonse = [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
narrative = [7234 1152   26  686   56    4    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
gt_response = [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_true = 1

utterance cnt = 2
resonse = [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
narrative = [7234 1152   26  686   56    4    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
gt_response = [7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [24]:
#browse_idx(dev_pad, 11)

In [25]:
# shape of embeddings (총단어수+1=43514, embedding_size=203)
print(embeddings.shape)

(43514, 200)


In [26]:
# embeddings의 첫번째 data는 0.으로 채워져 있다. PADDING용 이다.
embeddings[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# Word2Vec

In [27]:
import numpy as np

EOS_ID = tokenizer.sep_token_id

positive_sessions = []
for unit_data in data_dict:
    unit_contexts = [tokenizer.tokenize(text) for text in unit_data['script'] + ['[SEP]'] ]
    unit_narrative = tokenizer.tokenize(unit_data['storyline'])
    positive_sessions.append([unit_contexts, unit_narrative, 1])
print("all suitable sessions: ", len(positive_sessions))

# reproducibility를 위한 random seed 설정
np.random.seed(42)
# random shuffle data
np.random.shuffle(positive_sessions)

all suitable sessions:  11655


In [28]:
train_num = int(len(positive_sessions) * 0.9)
dev_test_num = int(len(positive_sessions) * 0.05)
train_sessions, dev_sessions, test_sessions = positive_sessions[:train_num], positive_sessions[train_num: train_num + dev_test_num], positive_sessions[train_num + dev_test_num:]
print('number of train =', len(train_sessions), ', val =', len(dev_sessions), ', test =', len(test_sessions))

number of train = 10489 , val = 582 , test = 584


In [29]:
train_texts = []
for train_session in train_sessions:
    train_texts += train_session[0]
    train_texts.append(train_session[1])
print('number of word2vec training sentences =', len(train_texts))

number of word2vec training sentences = 144229


In [30]:
train_sessions[0]

[[['불안',
   '##해',
   '하',
   '##는',
   '미치',
   '가족',
   '##들',
   ',',
   '멜라',
   '##니',
   '##가',
   '앨리스',
   '##를',
   '다독',
   '##인다',
   '.'],
  ['창문',
   '##을',
   '확인',
   '##하',
   '##는',
   '미치',
   '멜라',
   '##니',
   '##가',
   '사',
   '##온',
   '잉',
   '##꼬',
   '##를',
   '보고',
   '착',
   '##잡',
   '##해',
   '##한다',
   '.'],
  ['불안', '##해', '하', '##는', '미치', '가족', '##과', '멜라', '##니', '.'],
  ['드디어', '새', '##들', '##의', '공격', '##이', '시작', '##된', '##다', '.'],
  ['팔',
   '##의',
   '상처',
   '##를',
   '입',
   '##은',
   '미치',
   '##가',
   '새',
   '##들',
   '##의',
   '공격',
   '##을',
   '맨',
   '##손',
   '##으로',
   '막아내',
   '##지만',
   '역부족',
   '##이다',
   '.'],
  ['새',
   '##들이',
   '떠나',
   '##고',
   '조용',
   '##해',
   '##지',
   '##자',
   '한',
   '숨',
   '돌리',
   '##는',
   '사람',
   '##들',
   '.'],
  ['[SEP]']],
 ['불안', '##해하', '##는', '멜라', '##니', '##와', '미치', '가족', '.'],
 1]

In [31]:
train_texts[5:8]

[['새',
  '##들이',
  '떠나',
  '##고',
  '조용',
  '##해',
  '##지',
  '##자',
  '한',
  '숨',
  '돌리',
  '##는',
  '사람',
  '##들',
  '.'],
 ['[SEP]'],
 ['불안', '##해하', '##는', '멜라', '##니', '##와', '미치', '가족', '.']]

In [32]:
from gensim.models import Word2Vec

# word2vec 학습
model = Word2Vec(sentences = train_texts, vector_size = 200, window = 7, min_count = 5, workers = 4)

**참조 링크**  
https://machinelearningmastery.com/develop-word-embeddings-python-gensim/  
https://wikidocs.net/50739  

In [33]:
print('total num of words =', len(model.wv.key_to_index))
print('first word = "%s"'%model.wv.index_to_key[0])
print('last word = "%s"'%model.wv.index_to_key[-1])

total num of words = 11882
first word = "."
last word = "디지털"


In [34]:
# word2vec이 잘 학습되었는지 여러가지 테스트를 수행하자.
print(model.wv.most_similar("가족"))

[('친척', 0.7296834588050842), ('친구', 0.7136985063552856), ('동생', 0.6769063472747803), ('딸', 0.6370465755462646), ('팀원', 0.6250258088111877), ('아들', 0.6217833161354065), ('식구', 0.6177724599838257), ('동료', 0.6113835573196411), ('동물', 0.6099558472633362), ('어른', 0.6072829365730286)]


In [35]:
key = '가족'
print(model.wv.key_to_index[key])
print(model.wv.get_vector(key)[:5])
key = '[UNK]'
print(model.wv.key_to_index[key])
print(model.wv.key_to_index.get('최민수', 0))

133
[-1.0027357   0.48399073  0.8626121   1.2091957   0.7915889 ]
753
0


In [36]:
print(model.wv.get_vector(0)[:2])
model.wv.get_vector('.')[:2]

[-0.17658584 -0.305841  ]


array([-0.17658584, -0.305841  ], dtype=float32)

## 데이터 저장

`embeddings.pkl`과 `vocab.txt`를 생성한다.

In [37]:
with open("./data/vocab.txt", "w", encoding="utf-8") as file:
    for i, key in enumerate(model.wv.index_to_key):
        file.write('%s\t%i\n'%(key, i))

In [38]:
import pickle

new_embeddings = np.array([[0.]*200],dtype='float32') 
for i in range(len(model.wv.index_to_key)):
    new_embeddings = np.append(new_embeddings, [model.wv.get_vector(i)], axis=0)

with open('data/embeddings_ko.pkl', 'wb') as f:
    pickle.dump(new_embeddings, f)

# 학습 데이터셋 준비

In [39]:
EOS_ID = model.wv.key_to_index['[SEP]']+1
UNK_ID = model.wv.key_to_index['[UNK]']+1

In [40]:
import pickle

vocab = {}
positive_data = []

with open("./data/vocab.txt", "r", encoding="utf-8") as fr:
    for idx, line in enumerate(fr):
        line = line.strip().split("\t")
        vocab[line[0]] = idx + 1

In [41]:
# sample id 출력 확인
vocab['가족']

134

**positive data 준비**

In [42]:
positive_data = []

for unit in positive_sessions:
    narrative = unit[1]
    #print(narrative)
    context = unit[0]
    narrative_id = [vocab.get(word, UNK_ID) for word in narrative]
    context_id = [[vocab.get(word, UNK_ID) for word in sent] for sent in context]
    if len(narrative_id) == 0 or len(context_id) == 0:
        continue
    data = [context_id, narrative_id, 1]
    positive_data.append(data)

In [43]:
dev_test_num = int(len(positive_data) * 0.05)
train, dev, test = positive_data[:train_num], positive_data[train_num: train_num + dev_test_num], positive_data[train_num + dev_test_num:]

In [44]:
import random
train_all, dev_all, test_all = [], [], []
for context_id, narrative_id, _ in train:
    num_context = len(context_id)
    for i in range(1, num_context):
        context = context_id[:i]
        response = context_id[i]
        train_all.append([context, response, narrative_id, response, 1])
        flag = True
        while flag:
            random_idx = random.randint(0, len(positive_data) - 1)
            random_context = positive_data[random_idx][0]
            random_idx_2 = random.randint(0, len(random_context) - 1)
            random_response = random_context[random_idx_2]
            if len(response) != len(random_response):
                flag = False
                train_all.append([context, random_response, narrative_id, response, 0])
            else:
                for idx, wid in enumerate(response):
                    if wid != random_response[idx]:
                        flag = False
                        train_all.append([context, random_response, narrative_id, response, 0])
                        break
print(train_all[0]) 
print(train_all[1])

[[[1154, 36, 26, 2, 1227, 134, 15, 24, 2119, 60, 10, 462, 4, 5485, 831, 1]], [945, 3, 342, 12, 2, 1227, 2119, 60, 10, 230, 294, 1699, 5439, 4, 77, 3416, 2321, 36, 14, 1], [1154, 979, 2, 2119, 60, 17, 1227, 134, 1], [945, 3, 342, 12, 2, 1227, 2119, 60, 10, 230, 294, 1699, 5439, 4, 77, 3416, 2321, 36, 14, 1], 1]
[[[1154, 36, 26, 2, 1227, 134, 15, 24, 2119, 60, 10, 462, 4, 5485, 831, 1]], [25], [1154, 979, 2, 2119, 60, 17, 1227, 134, 1], [945, 3, 342, 12, 2, 1227, 2119, 60, 10, 230, 294, 1699, 5439, 4, 77, 3416, 2321, 36, 14, 1], 0]


In [45]:
for context_id, narrative_id, _ in dev:
    num_context = len(context_id)
    for i in range(1, num_context):
        context = context_id[:i]
        response = context_id[i]
        dev_all.append([context, response, narrative_id, response, 1])
        count = 0
        negative_samples = []
        # fix count 버그
        while count < 8:
            random_idx = random.randint(0, len(positive_data) - 1)
            random_context = positive_data[random_idx][0]
            random_idx_2 = random.randint(0, len(random_context) - 1)
            random_response = random_context[random_idx_2]
            if random_response not in negative_samples and random_response != [EOS_ID]:
                if len(response) != len(random_response):
                    dev_all.append([context, random_response, narrative_id, response, 0])
                    count += 1
                    negative_samples.append(random_response)
                else:
                    for idx, wid in enumerate(response):
                        if wid != random_response[idx]:
                            dev_all.append([context, random_response, narrative_id, response, 0])
                            count += 1
                            negative_samples.append(random_response)
                            break
        if response == [EOS_ID]:
            dev_all.append([context, [EOS_ID], narrative_id, response, 1])
        else:
            dev_all.append([context, [EOS_ID], narrative_id, response, 0])
print(dev_all[0], dev_all[1], dev_all[2])

[[[742, 6671, 29, 591, 274, 212, 6, 6655, 8, 147, 1]], [28, 2, 942, 368, 3, 217, 4, 35, 1], [126, 13, 212, 19, 536, 3, 754, 123, 1073, 893, 3, 364, 9, 20, 11, 1, 7838, 599, 676, 38, 28, 2, 1476, 521, 3, 3133, 47, 20, 11, 1, 526, 2, 210, 126, 6, 1162, 23, 72, 89, 103, 11, 1], [28, 2, 942, 368, 3, 217, 4, 35, 1], 1] [[[742, 6671, 29, 591, 274, 212, 6, 6655, 8, 147, 1]], [1792, 4, 848, 54, 639, 10, 268, 4, 3640, 47, 343, 1003, 13, 2156, 5866, 4, 2075, 1], [126, 13, 212, 19, 536, 3, 754, 123, 1073, 893, 3, 364, 9, 20, 11, 1, 7838, 599, 676, 38, 28, 2, 1476, 521, 3, 3133, 47, 20, 11, 1, 526, 2, 210, 126, 6, 1162, 23, 72, 89, 103, 11, 1], [28, 2, 942, 368, 3, 217, 4, 35, 1], 0] [[[742, 6671, 29, 591, 274, 212, 6, 6655, 8, 147, 1]], [11600, 423, 5, 239, 734, 15, 3, 230, 2, 363, 688, 23, 230, 90, 11, 1], [126, 13, 212, 19, 536, 3, 754, 123, 1073, 893, 3, 364, 9, 20, 11, 1, 7838, 599, 676, 38, 28, 2, 1476, 521, 3, 3133, 47, 20, 11, 1, 526, 2, 210, 126, 6, 1162, 23, 72, 89, 103, 11, 1], [28, 2, 

In [46]:
for context_id, narrative_id, _ in test:
    num_context = len(context_id)
    for i in range(1, num_context):
        context = context_id[:i]
        response = context_id[i]
        test_all.append([context, response, narrative_id, response, 1])
        count = 0
        negative_samples = []
        # fix count 버그
        while count < 8:
            random_idx = random.randint(0, len(positive_data) - 1)
            random_context = positive_data[random_idx][0]
            random_idx_2 = random.randint(0, len(random_context) - 1)
            random_response = random_context[random_idx_2]
            if random_response not in negative_samples and random_response != [EOS_ID]:
                if len(response) != len(random_response):
                    test_all.append([context, random_response, narrative_id, response, 0])
                    negative_samples.append(random_response)
                    count += 1
                else:
                    for idx, id in enumerate(response):
                        if id != random_response[idx]:
                            test_all.append([context, random_response, narrative_id, response, 0])
                            negative_samples.append(random_response)
                            count += 1
                            break
        if response == [EOS_ID]:
            test_all.append([context, [EOS_ID], narrative_id, response, 1])
        else:
            test_all.append([context, [EOS_ID], narrative_id, response, 0])
print(test_all[0], test_all[1], test_all[2])

[[[261, 2, 1410, 5, 7, 32, 6, 286, 3, 727, 14, 1]], [28, 2, 1410, 4, 373, 9, 20, 11, 1], [261, 2, 709, 2266, 23, 961, 8, 1360, 1, 96, 509, 95, 7, 191, 416, 404, 1042, 2, 261, 4, 1528, 36, 250, 1], [28, 2, 1410, 4, 373, 9, 20, 11, 1], 1] [[[261, 2, 1410, 5, 7, 32, 6, 286, 3, 727, 14, 1]], [79, 41, 57, 192, 19, 6449, 101, 2, 64, 1], [261, 2, 709, 2266, 23, 961, 8, 1360, 1, 96, 509, 95, 7, 191, 416, 404, 1042, 2, 261, 4, 1528, 36, 250, 1], [28, 2, 1410, 4, 373, 9, 20, 11, 1], 0] [[[261, 2, 1410, 5, 7, 32, 6, 286, 3, 727, 14, 1]], [408, 2, 180, 5, 1258, 564, 418, 16, 5556, 7, 2736, 389, 11, 1], [261, 2, 709, 2266, 23, 961, 8, 1360, 1, 96, 509, 95, 7, 191, 416, 404, 1042, 2, 261, 4, 1528, 36, 250, 1], [28, 2, 1410, 4, 373, 9, 20, 11, 1], 0]


In [47]:
print('total train count =', len(train_all))
print('total val count =', len(dev_all))
print('total test count =', len(test_all))

total train count = 246578
total val count = 66430
total test count = 66830


In [48]:
def get_numpy_from_nonfixed_2d_array(aa, max_sentence_len=50, max_num_utterance=10, padding_value=0):
    PAD_SEQUENCE = np.array([0] * max_sentence_len)
    rows = np.empty([0, max_sentence_len], dtype='int')
    aa = aa[:max_num_utterance]
    for a in aa:
        sentence_len = len(a)
        if sentence_len < max_sentence_len:
            rows  = np.append(rows, [np.pad(a, (0, max_sentence_len-sentence_len), 'constant', constant_values=padding_value)[:max_sentence_len]], axis=0)
        else:
            rows = np.append(rows, [a[:max_sentence_len]], axis=0)
    num_utterance = len(aa)
    if num_utterance < max_num_utterance:
        rows = np.append(rows, [PAD_SEQUENCE]*(max_num_utterance-num_utterance), axis=0)
    # add empty +1 sentence
    rows = np.append(rows, [PAD_SEQUENCE], axis=0)
    #return np.concatenate(rows, axis=0).reshape(-1, max_sentence_len)
    return rows

def get_numpy_from_nonfixed_1d_array(a, max_sentence_len=50, padding_value=0):
    sentence_len = len(a)
    if sentence_len < max_sentence_len:
        return np.pad(a, (0, max_sentence_len-sentence_len), 'constant', constant_values=padding_value)
    else:
        return np.array(a[:max_sentence_len])

cc_test_data = [
        [1,2],
        [4,5,6],
        [7]
     ]
get_numpy_from_nonfixed_2d_array(cc_test_data, max_sentence_len=5, max_num_utterance=4)

array([[1, 2, 0, 0, 0],
       [4, 5, 6, 0, 0],
       [7, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [49]:
get_numpy_from_nonfixed_1d_array([1,2])

array([1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [50]:
from tqdm import tqdm
def pad_process(data, max_sentence_len=50, max_num_utterance=10):
    utterance = []
    response = []
    narrative = []
    gt_response = []
    y_true = []
    for unit in tqdm(data):
        utterance.append(get_numpy_from_nonfixed_2d_array(unit[0]))
        response.append(get_numpy_from_nonfixed_1d_array(unit[1]))
        narrative.append(get_numpy_from_nonfixed_1d_array(unit[2]))
        gt_response.append(get_numpy_from_nonfixed_1d_array(unit[3]))
        y_true.append(unit[4])
        
    utterance = np.stack(utterance)
    response = np.stack(response)
    narrative = np.stack(narrative)
    gt_response = np.stack(gt_response)
    y_true = np.stack(y_true)
    return (utterance, response, narrative, gt_response, y_true)

train_pad = pad_process(train_all)
dev_pad = pad_process(dev_all)
test_pad = pad_process(test_all)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66830/66830 [00:12<00:00, 5213.71it/s]


**학습데이터셋 저장**

In [51]:
with open('data/train_ko.pkl', 'wb') as f:
    pickle.dump(train_pad, f)
with open('data/dev_ko.pkl', 'wb') as f:
    pickle.dump(dev_pad, f)
with open('data/test_ko.pkl', 'wb') as f:
    pickle.dump(test_pad, f)

In [106]:
with open('data/positive_ko.pkl', "wb") as f:
    pickle.dump(positive_data, f)

In [107]:
model.save("data/word2vec.model")

In [109]:
model2 = Word2Vec.load("data/word2vec.model")

In [110]:
#print(model.wv.index_to_key[2])
for data in train[1:2]:
    utterances = data[0]
    narrative = data[1]
    tokens = [model2.wv.index_to_key[i-1] for i in narrative]
    output = tokenizer.convert_tokens_to_string(tokens)
    print(output)
    break

처음으로 베르메르의 집에 발을 들여놓는다.
